In [2]:
import sounddevice as sd
import soundfile as sf
import whisper
import tempfile

# Parameters for recording
duration = 5  # seconds
fs = 44100    # sampling rate (Hz)

print(f"Recording for {duration} seconds... Please speak now!")

# Record audio from the default microphone
recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
sd.wait()  # Wait until recording is finished

# Save the recorded audio to a temporary WAV file
with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio:
    filename = temp_audio.name
    sf.write(filename, recording, fs)
print("Recording complete. Transcribing...")

# Load the Whisper model (choose an appropriate size: tiny, base, small, medium, large)
model = whisper.load_model("base")

# Transcribe the audio file
result = model.transcribe(filename)
print("Transcription:", result["text"])

Recording for 5 seconds... Please speak now!
Recording complete. Transcribing...


100%|███████████████████████████████████████| 139M/139M [00:41<00:00, 3.49MiB/s]
e:\langflow_directory\langchain_venv\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues relate

Transcription:  Um, hey.


In [3]:
import sounddevice as sd
import soundfile as sf
import whisper
import tempfile
import time
import os

# Load the Whisper model (choose an appropriate size)
model = whisper.load_model("base")

# Recording parameters
fs = 44100    # Sampling rate
duration = 3  # Duration of each segment in seconds

print("Real-time transcription started. Speak into your microphone (Press Ctrl+C to stop)...")

try:
    while True:
        # Record a short segment
        recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype="float32")
        sd.wait()  # Wait until recording is complete
        
        # Save the recorded segment to a temporary WAV file
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio:
            filename = temp_audio.name
            sf.write(filename, recording, fs)
        
        # Transcribe the audio segment using Whisper
        result = model.transcribe(filename)
        print("Transcription:", result["text"])

        # Optionally remove the temporary file
        os.remove(filename)
        
        # Small pause before next recording (if needed)
        time.sleep(0.1)

except KeyboardInterrupt:
    print("\nReal-time transcription stopped.")

Real-time transcription started. Speak into your microphone (Press Ctrl+C to stop)...
Transcription: 
Transcription:  I don't know where the trans...
Transcription:  yeah
Transcription: 
Transcription: 
Transcription:  Yeah, okay cool.
Transcription:  So now...
Transcription: 
Transcription: 

Real-time transcription stopped.


In [ ]:
import threading
import time
import sounddevice as sd
import soundfile as sf
import whisper
from threading import Event

# Recording parameters
fs = 44100          # Sampling rate (Hz)
duration = 3        # Duration of each segment (seconds)

# Load the Whisper model (choose an appropriate size: tiny, base, small, etc.)
model = whisper.load_model("base")

# Define two persistent filenames to alternate recording
filenames = ["audio1.wav", "audio2.wav"]
# Events to signal when a buffer has been recorded and saved
record_ready = [Event(), Event()]

def record_loop(buffer_index):
    """Continuously record into a given buffer file and signal when ready."""
    while True:
        print(f"Recording into {filenames[buffer_index]} ...")
        recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype="float32")
        sd.wait()  # Wait until recording is finished
        sf.write(filenames[buffer_index], recording, fs)
        record_ready[buffer_index].set()  # Signal that the recording is ready
        # Allow a small pause to avoid overlap issues before next recording
        time.sleep(0.1)

# Start two background threads, one for each buffer file
for i in range(2):
    t = threading.Thread(target=record_loop, args=(i,), daemon=True)
    t.start()

print("Real-time transcription started. Speak into your microphone (Press Ctrl+C to stop)...")
buffer_index = 0

try:
    while True:
        # Wait until the current buffer’s recording is available
        if record_ready[buffer_index].wait(timeout=duration+1):
            # Transcribe the recorded audio from the persistent file
            filename = filenames[buffer_index]
            result = model.transcribe(filename)
            print(f"Buffer {buffer_index} transcription: {result['text']}")
            # Clear the event to prepare for the next recording into this buffer
            record_ready[buffer_index].clear()
        else:
            print(f"No new recording available for buffer {buffer_index}.")
        # Rotate between the two buffers
        buffer_index = (buffer_index + 1) % 2

except KeyboardInterrupt:
    print("\nReal-time transcription stopped.")

Recording into audio1.wav ...
Recording into audio2.wav ...
Real-time transcription started. Speak into your microphone (Press Ctrl+C to stop)...
Recording into audio1.wav ...Recording into audio2.wav ...

Buffer 0 transcription:  so I think this
Recording into audio2.wav ...Recording into audio1.wav ...

Buffer 1 transcription:  so I think this
Recording into audio2.wav ...Recording into audio1.wav ...

Buffer 0 transcription:  real time audio is being
Recording into audio2.wav ...
Recording into audio1.wav ...
Recording into audio1.wav ...Recording into audio2.wav ...

Recording into audio2.wav ...Recording into audio1.wav ...

Buffer 1 transcription:  written right........
Recording into audio1.wav ...Recording into audio2.wav ...

Buffer 0 transcription: 
Recording into audio1.wav ...Recording into audio2.wav ...

Recording into audio1.wav ...Recording into audio2.wav ...

Recording into audio2.wav ...Recording into audio1.wav ...

Recording into audio1.wav ...Recording into audio2

Recording into audio2.wav ...
Recording into audio1.wav ...
Recording into audio1.wav ...Recording into audio2.wav ...

Recording into audio1.wav ...Recording into audio2.wav ...

Recording into audio2.wav ...
Recording into audio1.wav ...
Recording into audio2.wav ...Recording into audio1.wav ...

Recording into audio1.wav ...Recording into audio2.wav ...

Recording into audio1.wav ...Recording into audio2.wav ...

Recording into audio2.wav ...
Recording into audio1.wav ...
Recording into audio1.wav ...
Recording into audio2.wav ...
Recording into audio1.wav ...Recording into audio2.wav ...

Recording into audio1.wav ...Recording into audio2.wav ...

Recording into audio2.wav ...Recording into audio1.wav ...

Recording into audio1.wav ...Recording into audio2.wav ...

Recording into audio1.wav ...Recording into audio2.wav ...

Recording into audio1.wav ...Recording into audio2.wav ...

Recording into audio2.wav ...Recording into audio1.wav ...

Recording into audio1.wav ...Recording i

: 